In [60]:
import boto3
import codecs
import csv
import datetime
import holidays
import json
import time

### To adjust the filter for dates in the USFR spider

In [6]:
def next_business_day(date, time_span, time_unit, country_code):
    ONE_DAY = datetime.timedelta(days=1)
    HOLIDAYS = holidays.CountryHoliday(country_code)
    next_day = date + ONE_DAY
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS:
       next_day += ONE_DAY
    return next_day

def create_date_list(start_date, to_date, time_span, time_unit, country_code):
    dates = []
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    to_date = datetime.datetime.strptime(to_date, '%Y-%m-%d').date()
    while start_date < to_date:
        start_date = next_business_day(start_date, time_span, time_unit, country_code)
        dates.append(start_date)
    return dates

In [21]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/Scrapy/scrapy_official_newspapers/output/"
file_name = "USFR_20210310_1.csv"
file = path + file_name

dates_dict = {}

with open(file, 'r', errors="surrogateescape") as f:   
    reader = csv.reader(f)
    for row in reader: #unicode(reader, errors='ignore'):
        dates_dict[datetime.datetime.strptime(row[6], '%Y-%m-%d').date()] = 0
        
from_date = "2010-01-01"
to_date = "2010-01-07"
country_code = "US"
for day in create_date_list(from_date, to_date, 1, "days", country_code):
    if day in dates_dict:
        print(day)

2010-01-04
2010-01-05


In [9]:
type(from_date)

str

### To remove duplicates from csv by file name

In [43]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/Scrapy/scrapy_official_newspapers/output/"
file_name = "India_20210310.csv"
file = path + file_name

no_dups_dict = {}

i = 0

with open(file, 'r', errors="surrogateescape") as f:  # 
    reader = csv.reader(f)
    for row in reader:
        i += 1
        no_dups_dict[row[3]] = row
        
print("* Number of entries:", i, "\n* Number of unique entries:", len(no_dups_dict))

file_name = "India_metadata.csv"
file = path + file_name
i = 0
with open(file, 'w') as f:
    writer = csv.writer(f)
    for key, value in no_dups_dict.items():
        value[0] = value[0].replace("\udca0", "")
        try:
            writer.writerow([ value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11]])
        except:
            print("* ", value[3], "could not be saved. ", value[0])

* Number of entries: 1954 
* Number of unique entries: 1040


### To upload csv form S3

In [46]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_WRI.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-nlp-policy"
region = 'us-east-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [56]:
import boto3
import codecs
import csv

# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_WRI.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-nlp-policy"
region = 'us-east-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

# Load file

# You can adapt it to get the whole contents of each row of the bucket updating a list or lists that replicates
# the csv 

# The counter just for testing purposes
i = 0

file_key = "metadata/USA_metadata.csv"

obj = s3.Object(bucket_name = s3BucketName, key = file_key)
for row in csv.reader(codecs.getreader("utf-8")(obj.get()['Body'])):
    if i < 5:
        print(row[3])
    i += 1


dd0ccf82ae888eca4c9926f3dcb393b85c0d6c76.txt
c69fd24b9d42100fe63a17f1e67ad35c6d1b4011.txt
cc1e1f9a78f07a43e7c4e80e15b4411cc4022794.txt
3a17da0f2708a1003d1f82bd77fdbc6a94b57b7e.txt
dff9b9e3d8fe809eb197c4d9ae96fd18a7d66810.txt


In [68]:


i = 0

file_key = "metadata/India_metadata.csv"
obj_key_dict = {}

obj = s3.Object(bucket_name = s3BucketName, key = file_key)
for row in csv.reader(codecs.getreader("utf-8")(obj.get()['Body'])):
    if i < 1001:
#         print(row[3])
        obj_key_dict["english_documents/text_files/" + row[3]] = 0
    i += 1
    
# j = 0
# k = 1
# Ti = time.perf_counter()
# for item in obj_key_dict:
#     try:
#         obj = s3.Object(bucket_name = s3BucketName, key = file_key)
#         file = obj.get()['Body'].read()
#         j += 1
#     except:
#         print(f'{k} ** the file {item} was not found')

# Tf = time.perf_counter()

# print(f"Accessing {j} objects from the S3 bucket has taken {Tf - Ti:0.4f} seconds")
j = 0
Ti = time.perf_counter()
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix='english_documents/text_files/'):
    if obj.key in  obj_key_dict:
#         file = obj.get()['Body'].read()
#         print(obj.key)
        j += 1
        
Tf = time.perf_counter()
print(f"Accessing {j} objects from the S3 bucket has taken {Tf - Ti:0.4f} seconds")

Accessing 0 objects from the S3 bucket has taken 9.8081 seconds


In [57]:
keyword = "dd0ccf82ae888eca4c9926f3dcb393b85c0d6c76"
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix="english_documents/text_files/"):
    if keyword in obj.key: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)

15594 ** english_documents/text_files/dd0ccf82ae888eca4c9926f3dcb393b85c0d6c76.txt
